In [ ]:
import nilearn
from nilearn.decomposition import CanICA
import scipy
from scipy import io
import h5py
import numpy as np
import nibabel as nib
import os
import pandas as pd

In [ ]:
nilearn.__version__

In [ ]:
data_root = "/hd2/research/Naturalistic/data/sans/"
code_root = "/hd2/research/Naturalistic/code/SANS/"

sub_list = pd.read_csv(code_root + 'files/sublist', header=None)
im_path = data_root + "sans_bfp_minimal_tensor.mat"

In [ ]:
# load grayordinate data: only need to run once
# ICA does not need synchronization
with h5py.File(im_path, 'r') as f:
    im = np.array(f['tensor'])

In [ ]:
print(im.shape)
print(im.dtype)

num_sub = im.shape[0]

In [ ]:
# if haven't sliced images
nib_file_list = []

for i in np.arange(num_sub):
    cur_im = im[i, :, :]
    cur_im = np.expand_dims(cur_im, [1, 2])
    cur_im_nib = nib.Nifti1Image(cur_im, affine=np.eye(4))
    cur_save_path = data_root + "ICA/input/tfMRI_11k_%d.nii" % (i + 1)
    nib.save(cur_im_nib, cur_save_path)
    nib_file_list.append(cur_save_path)
#     print(sub)

nib_file_list

In [ ]:
# if images already sliced

nib_file_list = []

for i in np.arange(num_sub):
    cur_save_path = rst_path + "ICA/input/" + "tfMRI_%s_7T_11k_%d.nii" % (movie_name_v1, i+1)
    nib_file_list.append(cur_save_path)
    
nib_file_list

In [ ]:
im1 = nib.load(data_root + 'ICA/input/tfMRI_11k_1.nii')
mask = np.ones(im1.shape[0:3]).astype(np.float32)
im1_mask = nib.Nifti1Image(mask, affine=np.eye(4))
print(mask.shape)
print(mask.dtype)

In [ ]:
canica = CanICA(mask=im1_mask, n_components=30, detrend=False, standardize=True, \
                threshold=None, mask_strategy=None, smoothing_fwhm=None, \
                n_init=10, verbose=True,
                memory="nilearn_cache", memory_level=2)
canica.fit(nib_file_list)

In [ ]:
trans = canica.transform(nib_file_list)

In [ ]:
canica_components_img = canica.components_img_
canica_components_img.shape

In [ ]:
canica_comp_arr = nilearn.image.get_data(canica_components_img)
canica_comp_new = np.squeeze(canica_comp_arr)
print(canica_comp_new.shape)

print(len(trans))
print(trans[0].shape)

scipy.io.savemat(data_root + "ICA/output/CanICA_decomp.mat", mdict={'spatial': canica_comp_new, 'temporal': trans})